# MNIST with tf.keras, tf.data, and [eager execution(知乎,必讀)](https://www.zhihu.com/question/67471378).

In this lab, you'll learn how create a `tf.data` [Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) for the MNIST dataset. The tf.data API provides tools for working with data (including common functionaltity, like shuffling, batching, etc), as well as high performance utilities (like parallel reads, and pre-fecthing to GPUs) - should you need these down the road. Here, we'll wrap MNIST with tf.data, just to introduce the API. Later, you'll learn how to use tf.data on your own data, say, to read a directory of images off disk.

To learn more about tf.data, you can check out [this](https://www.youtube.com/watch?v=EHHdyM3NNiA&index=26&list=PLQY2H8rRoyvxjVx3zfw4vA4cvlKogyLNN) talk from the [2018 TensorFlow Developer Summit](https://www.tensorflow.org/dev-summit/).

The rest of the code similar to the previous notebook, with three changes. 

1) Instead of using `model.fit`, we use `model.train_on_batch` as we iterate over our dataset. 
2) We will create a TensorFlow optimizer to pass to the Keras model.
3) We will enable eager execution.

### Eager execution
Eager execution is a mode for running TensorFlow that works just like regular Python _(instead of building graphs and session.run it)_. To learn more about eager, check out [this talk](https://www.youtube.com/watch?v=T8AW0fKP0Hs&list=PLQY2H8rRoyvxjVx3zfw4vA4cvlKogyLNN&index=8).

In [1]:
import peforth
%f : unknown py> getattr(sys.modules['__main__'],pop(),"Ûnk̂nôwn̂") py> str(tos())=='Ûnk̂nôwn̂' if drop false else true then ; // ( token -- thing Y|N) try to find it in __main__

In [2]:
# !pip install -q -U tensorflow==1.8.0
import tensorflow as tf

# Enable eager execution
tf.enable_eager_execution()

import numpy as np

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

In [4]:
TRAINING_SIZE = len(train_images)
TEST_SIZE = len(test_images)

# Reshape from (N, 28, 28) to (N, 784)
train_images = np.reshape(train_images, (TRAINING_SIZE, 784))
test_images = np.reshape(test_images, (TEST_SIZE, 784))

# Convert the array to float32 as opposed to uint8
train_images = train_images.astype(np.float32)
test_images = test_images.astype(np.float32)

# Convert the pixel values from integers between 0 and 255 to floats between 0 and 1
# 這個很重要，不知何故，感覺好像沒差，結果雖然有收斂，可是很慢，好像半途就下不去了
train_images /= 255
test_images /=  255

In [5]:
NUM_DIGITS = 10

print("Before", train_labels[0]) # The format of the labels before conversion

train_labels  = tf.keras.utils.to_categorical(train_labels, NUM_DIGITS)

print("After", train_labels[0]) # The format of the labels after conversion

test_labels = tf.keras.utils.to_categorical(test_labels, NUM_DIGITS)

Before 5
After [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]


In [6]:
# Cast the labels to floats, needed later
train_labels = train_labels.astype(np.float32)
test_labels = test_labels.astype(np.float32)

In [7]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(784,)))
model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))


# Create a TensorFlow optimizer, rather than using the Keras version
# This is currently necessary when working in eager mode
optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001)

# We will now compile and print out a summary of our model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


### Step 1) Create a tf.data Dataset

Here, we'll use the `tf.data.Dataset` [API](https://www.tensorflow.org/api_docs/python/tf/data) to convert the Numpy arrays into a TensorFlow dataset.

Next, we will create a simple for loop that will serve as our introduction into creating custom training loops. Although this essentially does the same thing as `model.fit` it allows us to get creative and customize the overall training process (should you like to, if you venture into research) and collect different metrics throughout the process.

In [8]:
BATCH_SIZE=128

# Because tf.data may work with potentially **large** collections of data
# we do not shuffle the entire dataset by default
# Instead, we maintain a buffer of SHUFFLE_SIZE elements
# and sample from there.
SHUFFLE_SIZE = 10000 

# Create the dataset
dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
dataset = dataset.shuffle(SHUFFLE_SIZE)
dataset = dataset.batch(BATCH_SIZE)

### Step 2) Iterate over the dataset
Here, we'll iterate over the dataset, and train our model using `model.train_on_batch`. To learn more about the elements returned from the dataset, you can print them out and try the `.numpy()` method.


In [11]:
EPOCHS=5

for epoch in range(EPOCHS):
  for images, labels in dataset:
    train_loss, train_accuracy = model.train_on_batch(images, labels)
  
  # Here you can gather any metrics or adjust your training parameters
  print('Epoch #%d\t Loss: %.6f\tAccuracy: %.6f' % (epoch + 1, train_loss, train_accuracy))
  

Epoch #1	 Loss: 8.730636	Accuracy: 0.458333
Epoch #2	 Loss: 9.066429	Accuracy: 0.437500
Epoch #3	 Loss: 9.905913	Accuracy: 0.385417
Epoch #4	 Loss: 8.562738	Accuracy: 0.468750
Epoch #5	 Loss: 8.730651	Accuracy: 0.458333


In [10]:
loss, accuracy = model.evaluate(test_images, test_labels)
print('Test accuracy: %.2f' % (accuracy))

10000/10000 [==============================] - 1s 89us/step
Test accuracy: 0.45


### Congratulations
You have trained a model on MNIST using Keras, eager execution, and tf.data.